## Day 23: Coprocessor Conflagration

You decide to head directly to the CPU and fix the printer from there. As you get close, you find an experimental coprocessor doing so much work that the local programs are afraid it will halt and catch fire. This would cause serious issues for the rest of the computer, so you head in and see what you can do.

### Part one

The code it's running seems to be a variant of the kind you saw recently on that tablet. The general functionality seems very similar, but some of the instructions are different:

 - set X Y sets register X to the value of Y.
 
 - sub X Y decreases register X by the value of Y.
 
 - mul X Y sets register X to the result of multiplying the value contained in register X by the value of Y.
 
 - jnz X Y jumps with an offset of the value of Y, but only if the value of X is not zero. (An offset of 2 skips the next instruction, an offset of -1 jumps to the previous instruction, and so on.)
 
    Only the instructions listed above are used. The eight registers here, named a through h, all start at 0.

In [125]:
# Just to simplify: each register defaults to zero and can be read as attributes

class Registers(dict):
    
    def __init__(self):
        for k in 'abcdefgh':
            self.__setitem__(k, 0)
    
    def __getattr__(self, k):
        if k in 'abcdefgh':
            return self[k]
        raise AttributeError('Register {} not found'.format(k))
        
registers = Registers()
assert registers['a'] == 0
assert registers.a == 0
registers['a'] = 3
assert registers.a == 3
print('ok')

ok


In [147]:
from collections import defaultdict

class Proc:
    
    def __init__(self):
        self.registers = Registers()
        self.pc = 0
        self.code = []
        self._clock = 0
        self.freq = None
        self.end_flag = False
        self.stats = defaultdict(int)
        self.kernel = {
            'set': self.op_set,
            'sub': self.op_sub,
            'mul': self.op_mul,
            'jnz': self.op_jnz,
            }

    def get_value(self, s):
        return self.registers[s] if s in 'abcdefgh' else int(s)
        
    def op_set(self, register, value):
        v = self.get_value(value)
        self.registers[register] = v
        return 1
    
    def op_sub(self, register, value):
        v = self.get_value(value)
        self.registers[register] = self.registers[register] - v
        return 1

    def op_mul(self, register, value):
        v = self.get_value(value)
        self.registers[register] = self.registers[register] * v
        return 1
                        
    def op_jnz(self, register, value):
        v = self.get_value(value)
        flag = self.get_value(register) != 0
        return v if flag else 1
        
    def exec(self, line):
        cod_op, *args = line.split(' ')
        self.stats[cod_op] += 1
        method = self.kernel[cod_op]
        return method(*args)
        
    def must_exit(self):
        return any([
            self.pc >= len(self.code),
            self.end_flag,
        ])
    
    def step(self, tron=False):
        line = self.code[self.pc]
        if tron:
            print('{:>4} {:>6} {} [{}]'.format(
                self._clock,
                self.pc,
                self.code[self.pc],
                self.dump()
                ),
                end=' > '
            )
        self.pc += self.exec(line)
        self._clock += 1
        if tron:
            print(self.dump())
    
    def load_program(self, lines):
        self.code = [_ for _ in lines]
        self.pc = 0
        
    def run(self, lines, tron=False):
        self.load_program(lines)
        while True:
            self.step(tron=tron)
            if self.must_exit():
                break
        if tron:
            print('{:>8} PC: {} | end execution'.format(
                self._clock,
                self.pc,
            ))
        
    def dump(self):
        buff = [
            '{:3}'.format(self.registers[k])
            for k in 'abcdefgh'
            ]
        return ' '.join(buff)
    
    def trace(self):
        print('a: {a} b: {b} c: {c} d: {d} e: {e} f: {f} g: {g} h:{h}'.format(
             **self.registers
             ))
        
p = Proc()
p.exec('set b 11')
p.exec('set a 3')
p.exec('mul a b')
assert p.registers.a == 33
assert p.registers.b == 11
p.exec('sub b 2')
assert p.registers.b == 9
assert p.exec('jnz b 2') == 2
p.trace()
print('ok')

a: 33 b: 9 c: 0 d: 0 e: 0 f: 0 g: 0 h:0
ok


The coprocessor is currently set to some kind of debug mode, which allows for testing, but prevents it from doing any meaningful work.

If you run the program (your puzzle input), **how many times is the mul instruction invoked**?

In [127]:
def load_input(filename):
    with open(filename, 'r') as f:
        lines = [l.strip() for l in f.readlines()]
    return lines
        
program = load_input('input.txt')
for l in program[0:3]:
    print(l)
print('...')
for l in program[-3:]:
    print(l)

set b 99
set c b
jnz a 2
...
jnz 1 3
sub b -17
jnz 1 -23


In [137]:
program[0:8]

['set b 99',
 'set c b',
 'jnz a 2',
 'jnz 1 5',
 'mul b 100',
 'sub b -100000',
 'set c b',
 'sub c -17000']

In [159]:
p = Proc()
p.registers['a'] =  1
p.run(program[0:8])
p.trace()

a: 1 b: 109900 c: 126900 d: 0 e: 0 f: 0 g: 0 h:0


In [171]:
p = Proc()
p.run(program)

print('Part one:', p.stats['mul'])
p.trace()

Part one: 9409
a: 0 b: 99 c: 99 d: 99 e: 99 f: 0 g: 0 h:1


In [129]:
p.registers

{'a': 0, 'b': 99, 'c': 99, 'd': 99, 'e': 99, 'f': 0, 'g': 0, 'h': 1}

### Part two

Now, it's time to fix the problem.

The debug mode switch is wired directly to register a. You flip the switch, which makes register a now start at 1 when the program is executed.

Immediately, the coprocessor begins to overheat. Whoever wrote this program obviously didn't choose a very efficient implementation. You'll need to optimize the program if it has any hope of completing before Santa needs that printer working.

The coprocessor's ultimate goal is to determine the final value left in register h once the program completes. Technically, if it had that... it wouldn't even need to run the program.

After setting register a to 1, if the program were to run to completion, **what value would be left in register h**?

In [75]:
class ProcTwo(Proc):
    def __init__(self):
        super().__init__()
        self.registers['a'] = 1

p = ProcTwo()
assert p.registers.a == 1
p.load_program(program)
for i in range(21000000000):
    p.step(tron=p.registers.g == p.registers.b)
    if p.registers.g == p.registers.b:
        print(p.pc, p._clock)
        break
    # if p.pc == 17: print('e:', p.registers.e)
#p.run(program)

   0      0 set b 99 [  1   0   0   0   0   0   0   0] >   1  99   0   0   0   0   0   0
13 439596


In [184]:
# Python Version 

a = b = c = d = e = f = g = h = 0
def trace():
    global a,b,c,d,e,f,g,h
    print('a:', a, 'b:', b, 'c:', c, 'd:', d, 'e:', e, 'f:', f, 'g:', g, 'h:', h)

a = 1
if a != 0:
    b = 109900
    c = 126900
else:
    b = 99
    c = 99
trace()
while True:
    f = 1
    d = 2
    trace()
    while d != b:
        e = 2
        while e != b:
            if d * e == b:
                print('Encontrado <c:{}>'.format(c), end=' ')
                f = 0
                print('d[{}] * e[{}] == b[{}]'.format(d, e, b))
            if f == 0:
                break
            e = e + 1
        if f == 0:
            break
        d = d + 1
    if f == 0:
        h = h + 1
    if b == c:
        print('break')
        break
    b = b + 17
trace()  


a: 1 b: 109900 c: 126900 d: 0 e: 0 f: 0 g: 0 h: 0
a: 1 b: 109900 c: 126900 d: 2 e: 0 f: 1 g: 0 h: 0
Encontrado <c:126900> d[2] * e[54950] == b[109900]
a: 1 b: 109917 c: 126900 d: 2 e: 54950 f: 1 g: 0 h: 1
Encontrado <c:126900> d[3] * e[36639] == b[109917]
a: 1 b: 109934 c: 126900 d: 2 e: 36639 f: 1 g: 0 h: 2
Encontrado <c:126900> d[2] * e[54967] == b[109934]
a: 1 b: 109951 c: 126900 d: 2 e: 54967 f: 1 g: 0 h: 3
Encontrado <c:126900> d[43] * e[2557] == b[109951]
a: 1 b: 109968 c: 126900 d: 2 e: 2557 f: 1 g: 0 h: 4
Encontrado <c:126900> d[2] * e[54984] == b[109968]
a: 1 b: 109985 c: 126900 d: 2 e: 54984 f: 1 g: 0 h: 5
Encontrado <c:126900> d[5] * e[21997] == b[109985]
a: 1 b: 110002 c: 126900 d: 2 e: 21997 f: 1 g: 0 h: 6
Encontrado <c:126900> d[2] * e[55001] == b[110002]
a: 1 b: 110019 c: 126900 d: 2 e: 55001 f: 1 g: 0 h: 7
Encontrado <c:126900> d[3] * e[36673] == b[110019]
a: 1 b: 110036 c: 126900 d: 2 e: 36673 f: 1 g: 0 h: 8
Encontrado <c:126900> d[2] * e[55018] == b[110036]
a: 1 b: 11

KeyboardInterrupt: 

In [214]:
f = 1
d = 2
b = 110291
counter = 0
while d < b:
    if d % 1702 == 0:
        print('d vale {}, b vale {}'.format(d, b))
    e = 2
    while e <= b//2:
        if d * e == b:
            print('Encontrado <c:{}>'.format(c), end=' ')
            print('d[{}] * e[{}] == b[{}]'.format(d, e, b))
            f = 0
        e = e + 1
        if f == 0:
            break
    d = d + 17
    if f == 0:
        break
    

d vale 1702, b vale 110291
d vale 30636, b vale 110291
d vale 59570, b vale 110291
d vale 88504, b vale 110291


In [215]:
d

110298

In [212]:
d, b,e

(2450, 110291, 37047)

In [169]:
d = 2
c = 99

b = 99
counter = 0
while d != b:
    e = 2
    while e != b:
        if d * e == b:
            print('Encontrado', end=' ')
            f = 0
            print('d[{}] * e[{}] == b[{}]'.format(d, e, b))
        e = e + 1
    d = d + 1
    counter += 1

Encontrado d[3] * e[33] == b[99]
Encontrado d[9] * e[11] == b[99]
Encontrado d[11] * e[9] == b[99]
Encontrado d[33] * e[3] == b[99]


> Program find the numbers that **are not primes** in the range 109900 ... 126900, in steps of size 17: 109900, 109917, 109934 ... 126900

In [223]:
import math

def is_prime(n):   
    if n % 2 == 0 and n > 2:          
        return False                  
    return all(n % i for i in range(3, int(math.sqrt(n)) + 1, 2))
    
acc = 0
for i in range(109900, 126901, 17):
    acc += 1 if not is_prime(i) else 0
print('Part 2:', acc)


Part 2: 913
